In [0]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm import tqdm

dtype = torch.float
device = torch.device("cuda:0")

In [0]:
inputs = list(map(lambda s: Variable(torch.Tensor([s])), [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
]))
targets = list(map(lambda s: Variable(torch.Tensor([s])), [
    [0],
    [1],
    [1],
    [0]
]))

<img src="https://lh3.google.com/u/0/d/1MviIaN8DI37JSgcsS77B0Sm1iGjJojDz=w2880-h1544-iv1" width="500px">

In [0]:
max_iter = 40000
learning_rate = 0.3

In [0]:
activation = nn.Sigmoid()

In [0]:
class Net(nn.Module):
  
  def __init__(self, learning_rate=1e-3):
    super(Net, self).__init__()
    self.inputSize = 2
    self.hiddenSize = 2
    self.outputSize = 1
    self.learning_rate = learning_rate
    # weights
    self.W1 = Variable(torch.randn(self.inputSize, self.hiddenSize), requires_grad=True)# 2 X 2 tensor
    self.W2 = Variable(torch.randn(self.outputSize, self.hiddenSize), requires_grad=True)# 1 X 2 tensor
    self.B1 = Variable(torch.randn(self.hiddenSize, 1), requires_grad=True)
    self.B2 = Variable(torch.randn(self.outputSize, 1), requires_grad=True)
    
  def forward(self, X):
    z1 = self.W1.mm(X)+self.B1
    a1 = activation(z1)
    z2 = self.W2.mm(a1)+self.B2
    a2 = activation(z2)
    return a2
  def backward(self, Loss):
    Loss.backward()
    with torch.no_grad():
      self.W2 -= (self.learning_rate * self.W2.grad)
      self.B2 -= (self.learning_rate * self.B2.grad)
      self.W1 -= (self.learning_rate * self.W1.grad)
      self.B1 -= (self.learning_rate * self.B1.grad)
      # Manually zero the gradients after updating weights
      self.W1.grad.zero_()
      self.W2.grad.zero_()
      self.B1.grad.zero_()
      self.B2.grad.zero_()
  def train(self, x, y):
    pred = self.forward(x)
    Loss = F.binary_cross_entropy(pred, y)
#     print(Loss.item())
    self.backward(Loss)

In [6]:
NN = Net(learning_rate)
NN.cuda(device)

Net()

In [7]:
for i in tqdm(range(0, max_iter)):
  for input, target in zip(inputs, targets):
    NN.train(input.view(input.shape[1], 1), target)

100%|██████████| 40000/40000 [01:27<00:00, 455.43it/s]


In [8]:
NN.forward(torch.tensor([
    [1],
    [1]
], dtype=dtype))

tensor([[0.4806]], grad_fn=<SigmoidBackward>)

In [0]:
class EasyNet(nn.Module):
  def __init__(self):
    super(EasyNet, self).__init__()
    self.linear1 = torch.nn.Linear(2, 2)
    self.linear2 = torch.nn.Linear(2, 1)
  def forward(self, x):
    z1 = self.linear1(x)
    a1 = activation(z1)
    z2 = self.linear2(a1)
    a2 = activation(z2)
    return a2

In [10]:
ENN_model = EasyNet()
ENN_model.cuda(device)

EasyNet(
  (linear1): Linear(in_features=2, out_features=2, bias=True)
  (linear2): Linear(in_features=2, out_features=1, bias=True)
)

In [0]:
optimizer = torch.optim.SGD(ENN_model.parameters(), lr=learning_rate)

In [0]:
loss_fn = torch.nn.MSELoss(reduction='sum')

In [13]:
for i in tqdm(range(0, max_iter)):
  for input, target in zip(inputs, targets):
    pred = ENN_model(input.to(device))
#     Loss = loss_fn(pred, target)
    Loss = F.binary_cross_entropy(pred, target.to(device))
    #print(Loss.item())
    Loss.backward()
    optimizer.step()
    optimizer.zero_grad()

100%|██████████| 40000/40000 [02:54<00:00, 228.58it/s]


In [14]:
ENN_model.forward(inputs[2].to(device))

tensor([[0.6432]], device='cuda:0', grad_fn=<SigmoidBackward>)